In [1]:
import random

from datasets import Dataset, load_dataset
from tqdm import tqdm

/home/rizka-amalia/.pyenv/versions/3.12.2/envs/training-and-fine-tuning-language-model-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Softmax Loss Function

In [2]:
train_dataset = load_dataset(
    "glue",
    "mnli",
    split="train"
).select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")
train_dataset[0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1}

In [3]:
val_stsb = load_dataset(
    "glue",
    "stsb",
    split="validation"
)
val_stsb[0]

{'sentence1': 'A man with a hard hat is dancing.',
 'sentence2': 'A man wearing a hard hat is dancing.',
 'label': 5.0,
 'idx': 0}

Cosine Similarity Loss Function

In [4]:
train_dataset = load_dataset(
    "glue",
    "mnli",
    split="train"
).select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

mapping = {2: 0, 1: 0, 0: 1}
train_dataset = Dataset.from_dict(
    {
        "sentence1": train_dataset["premise"],
        "sentence2": train_dataset["hypothesis"],
        "label": [float(mapping[label]) for label in train_dataset["label"]]
    }
)
train_dataset[0]

{'sentence1': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'sentence2': 'Product and geography are what make cream skimming work. ',
 'label': 0.0}

Multiple Negatives Ranking Loss Function

In [5]:
mnli = load_dataset(
    "glue",
    "mnli",
    split="train"
).select(range(50_000))
mnli = mnli.remove_columns("idx")
mnli = mnli.filter(lambda x: True if x["label"] == 0 else False)

train_dataset = {
    "anchor": [],
    "positive": [],
    "negative": []
}
soft_negatives = mnli["hypothesis"]
random.shuffle(soft_negatives)
for row, soft_negatives in tqdm(zip(mnli, soft_negatives)):
    train_dataset["anchor"].append(row["premise"])
    train_dataset["positive"].append(row["hypothesis"])
    train_dataset["negative"].append(soft_negatives)
train_dataset = Dataset.from_dict(train_dataset)
train_dataset[0]

16875it [00:00, 26588.53it/s]


{'anchor': 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him',
 'positive': 'You lose the things to the following level if the people recall.',
 'negative': 'She pulled back at the last moment.'}